# Weighted Average

* https://docs.xarray.dev/en/stable/examples/area_weighted_temperature.html
* https://docs.xarray.dev/en/stable/generated/xarray.DataArray.weighted.html
* https://towardsdatascience.com/xarray-recipes-for-earth-scientists-c12a10c6a293#c729
* https://towardsdatascience.com/the-correct-way-to-average-the-globe-92ceecd172b7
* https://xcdat.readthedocs.io/en/stable/generated/xarray.Dataset.spatial.average.html
* https://xcdat.readthedocs.io/en/stable/examples/spatial-average.html

In [ ]:
import cf_xarray as cfxr
import xarray as xr
import pandas as pd
import numpy as np

## search intake catalog

In [ ]:
import intake

In [ ]:
cat_url = "https://raw.githubusercontent.com/cp4cds/c3s_34g_manifests/master/intake/catalogs/c3s.yaml"

cat = intake.open_catalog(cat_url)
list(cat)

In [ ]:
df_cmip6 = cat['c3s-cmip6'].read()
df_cmip6.head()

In [ ]:
df = df_cmip6.loc[
    (df_cmip6.experiment_id=="ssp245")
    & (df_cmip6.activity_id=="ScenarioMIP")
    & (df_cmip6.institution_id=="MPI-M")
    & (df_cmip6.member_id=="r1i1p1f1")
    & (df_cmip6.table_id=="Amon")
    & (df_cmip6.source_id=="MPI-ESM1-2-LR")
    & (df_cmip6.variable_id=="tasmax")
]
df.head()

In [ ]:
ds_path = list(set(df.path))[0]
ds_path

In [ ]:
# ds_url = f"https://data.mips.copernicus-climate.eu/thredds/fileServer/esg_c3s-cmip6/{ds_path}"
ds_url = f"http://esgf3.dkrz.de/thredds/fileServer/cmip6/{ds_path}"
ds_url

## download test data

In [ ]:
from pathlib import Path
data_dir = Path("/tmp")

In [ ]:
!wget $ds_url -O /tmp/test.nc

In [ ]:
nc_files = sorted(data_dir.glob('**/test.nc'))
nc_files

## calculate weighted average

In [ ]:
xr.set_options(keep_attrs=True)

ds = xr.open_dataset(nc_files[0])
ds

In [ ]:
ds['time'] = ds['time'].astype('int64')
ds['time_bnds'] = ds['time_bnds'].astype('int64')
ds

In [ ]:
weights = np.cos(np.deg2rad(ds.lat))
weights.name = "weights"
weights.fillna(0)
weights

In [ ]:
ds_weighted = ds.weighted(weights)
ds_weighted

In [ ]:
weighted_mean = ds_weighted.mean(("lon", "lat"))
weighted_mean